# PRACTICA GUIADA 2: Pivot Tables

* Una pivot table esuna operación similar a un `GroupBy` que suele ser común en planillas de cálculo y otros programas que operan con datos tabulares.
* Una pivot table toma una o varias columnas como input y agrupa las entradas en una tabla bidimensional que provee un resumen (generalmente, una agregación de los datos).

## Usando Pivot Tables

* Usaremos el dataset de pasajeros en el Titanic.
* El mismo contiene información sociodemográfica acerca de los pasajeros del barco (incluyendo, sexo, edad, clase de embarque, etc.)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [ ]:
titanic.head()

## Pivot Tables a mano

* Comencemos agrupando de acuerdo al género, si sobrevieron o no, etc.
* Esto podríamos hacerlo con un ``GroupBy``; veamos, por ejemplo, la proporción de sobrevivientes según sexo:

In [ ]:
titanic.groupby('sex')[['survived']].mean()

* Parece que 3 de cada 4 mujeres sobrevivieron, mientras que esta relación es notablemente menor entre los hombres (1 de cada 5)
* Veamos qué pasa si analizamos la sobrevivencia por sexo y clase:
    - hacemos un group por clase y género, seleccionamos los sobrevimientos, aplicamos una agregación (media) y combinamos los grupos resultantes y luego, "unstackeamos" el índice jerárquico:

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

* Esto da una idea mejor de cómo el género y la clase afectan las probabilidades de sobreviviencia. El problema es que el código empieza a parece un poco desorndeado y poco sencillo para leer.
* Este tipo de ``GroupBy`` es muy común en Pandas, por eso se incluyó un método ``pivot_table`` que maneja fácilmente este tipo de agregaciones multidimensionales.

## Sintaxis de las Pivot Table

* Veamos un equivalente de la operación previa usando el método ``pivot_table``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class')

* Es claramente más legible que ``groupby`` anterior.
* Como era esperable, había mayores probabildades (tanto entre hombres como mujeres) de sobrevivir si la persona provenía de clase alta.
* Las mujeres de la primera clase sobrevivieron casi en su totalidad (seguramente, por aquello de "las mujeres y los niños -con plata- primero").

### Multi-level pivot tables

* Igual que en ``GroupBy``, la operación de grouping  puede ser especificada con múltiples niveles.
* Por ejemplo, podríamos estar interesados en usar grupos de edad como tercera dimensión
* Generamos bins de la variable 'age' usando ``pd.cut``:

In [ ]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

* Podemos usar la misma estrategia cuando trabajamos con las columnas: agreguemos información acerca de la tarifa (fare) pagada, usando ``pd.qcut`` para computar cuantiles de forma automática:

In [ ]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

### Opcionales adicionales en Pivot Tables

* Todos los argumentos del método ``pivot_table`` son los siguientes:

```python
# call signature as of Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

* Vimos ejemplos de los primeros tres argumentos.
* ``fill_value`` y ``dropna``, se vinculan con la existencia de datos perdidos y son una forma relativamente simple de lidiar con ellos (volveremos sobre estos ejemplos más adelante).

* ``aggfunc`` controla el tipo de agregación que es aplicado (por defecto es una media)
* Al igual que con ``GroupBy``, la especificación de la operación de agregación tiene muchas opciones relativamete comunes (``'sum'``, ``'mean'``, ``'count'``, ``'min'``, ``'max'``, etc.) o bien alguna función que implementa una agregación (e.g., ``np.sum()``, ``min()``, ``sum()``, etc.).
* Además, puede expecificarse un diccionario que mapee una columna con alguna operación:

In [ ]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

* En este caso, se omitión la especificación de ``values``: al especificar un mapeo para ``aggfunc``, `values` se determina automáticamente.

* A veces es útil computar totlaes a lo largo de cada grupo: podemos hacerlo usando ``margins``:

In [ ]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

* La etiqueta del margen puede ser especificada con ``margins_name``, por defecto es ``"All"``.